In [4]:
pip install pandas boto3 pycountry


In [ ]:
import boto3
import pandas as pd
import boto3
from io import StringIO

In [7]:
s3 = boto3.client('s3',
    aws_access_key_id='*********',
    aws_secret_access_key='*********',
    region_name='us-east-1'
)

bucket = 'covid-staging-bucket-jayanthi'
key = 'raw/covid_testing_data_states_daily.csv'  # Adjust path

# Read from S3
obj = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(obj['Body'])
df.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [46]:
df['date'] = pd.to_datetime(df['date'])
df_cleaned = df[['fips', 'state', 'date', 'positive', 'recovered', 'death']]

# Drop rows with nulls
df_cleaned = df_cleaned.dropna()

# Type conversion
df_cleaned['positive'] = df_cleaned['positive'].astype(int)
df_cleaned['recovered'] = df_cleaned['recovered'].round(0).astype(int)
df_cleaned['death'] = df_cleaned['death'].round(0).astype(int)

    


In [52]:
csv_buffer = StringIO()
df_cleaned.to_csv(csv_buffer, index=False)

s3.put_object(
    Bucket=bucket,
    Key='output/factscovid_cleaned.csv',
    Body=csv_buffer.getvalue()
)

print("✅ Uploaded cleaned data to S3!")


✅ Uploaded cleaned data to S3!


In [13]:
hospital_df = load_csv_from_s3(bucket, 'raw/rearc_usa_hospital_beds.csv')  # update path if needed

dimhospital = hospital_df[[
    'FIPS', 'HOSPITAL_NAME', 'HOSPITAL_TYPE',
    'HQ_ADDRESS', 'HQ_CITY', 'HQ_STATE',
    'COUNTY_NAME', 'latitude', 'longtitude'
]].dropna().rename(columns=str.lower)


In [19]:
from io import StringIO

# Create in-memory CSV
csv_buffer = StringIO()

# Convert DataFrame to CSV format
dimhospital.to_csv(csv_buffer, index=False)

# Upload to S3
s3.put_object(
    Bucket='covid-staging-bucket-jayanthi',
    Key='output/dimhospital.csv',
    Body=csv_buffer.getvalue()
)

print("✅ dimhospital.csv uploaded to S3 successfully!")


✅ dimhospital.csv uploaded to S3 successfully!


In [27]:
print(jhud_df.columns.tolist())


['state_fips', 'state_name', 'lat', 'long', 'date', 'cases', 'deaths', 'tests_positive', 'tests_negative', 'tests_pending', 'tests', 'patients_icu', 'patients_hosp', 'patients_vent', 'recovered']


In [30]:
import pycountry
from io import StringIO

def get_country(state_name):
    if pd.isna(state_name):
        return None
    for subdivision in pycountry.subdivisions:
        if subdivision.name.lower() == state_name.lower():
            return pycountry.countries.get(alpha_2=subdivision.country_code).name
    return "United States"

# Select and rename relevant columns
dimregion = jhud_df[['state_fips', 'state_name', 'lat', 'long']].copy()
dimregion.columns = ['fips', 'province_state', 'latitude', 'longitude']

# Add country info
dimregion['country'] = dimregion['province_state'].apply(get_country)

# Drop any rows with nulls (optional)
dimregion = dimregion.dropna()

# Upload to S3
buffer = StringIO()
dimregion.to_csv(buffer, index=False)
s3.put_object(Bucket=bucket, Key='output/dimregion.csv', Body=buffer.getvalue())

print("✅ dimregion.csv uploaded to S3 successfully!")


✅ dimregion.csv uploaded to S3 successfully!


In [34]:
dimdate = pd.DataFrame({
    'date': df['date'],
    'day': df['date'].dt.day,
    'month': df['date'].dt.month,
    'year': df['date'].dt.year,
    'is_weekend': df['date'].dt.weekday >= 5
}).drop_duplicates()

buffer = StringIO()
dimdate.to_csv(buffer, index=False)
s3.put_object(Bucket=bucket, Key='output/dimdate.csv', Body=buffer.getvalue())

print("\n✅ dimdate.csv transformed and uploaded to S3!")



✅ dimdate.csv transformed and uploaded to S3!
